In [ ]:
#meta 2/12/2021 Kaggle Cassava Module 2-Inference Mini K-fold(for rapid iteration)
#fast.ai 2020 study group Kaggle Competition
#src https://www.kaggle.com/c/cassava-leaf-disease-classification
#prev competition src https://www.kaggle.com/c/cassava-disease
#references https://arxiv.org/pdf/1908.02900.pdf

#input: export.pkl
#output: submission.csv
#pwd: /kaggle/working
#summary: Export a model and submit predictions on mini data.
#based on version of my-kaggle-cassava-2-infer.ipynb -> my-kaggle-cassava-2-infer-mini_score0822.ipynb and
#based on version of my-kaggle-cassava-1-model-kfold.ipynb -> my-kaggle-cassava-1-model-mini-kfold_score0825.ipynb


#prev in my-kaggle-cassava-2-infer-mini_score0822.ipynb
#BEST 2/7+/2020 MODEL 2-INFERENCE MINI
#      $params: learn = load_learner($path_model, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25)
#               [not preds, _ = learn_inf.tta(dl=test_dl, n=10, beta=0.75, use_max=True)]
#      Score: 0.822, 0.824  GPU submission time: <1 min Rank: n/a (see Minii/my-kaggle-cassava-2-infer-mini_score0822.ipynb)
#      Score better: 0.835, 0.840 with bs=32 models, GPU submission time: <1 min Rank: n/a (no GH)
#      Note: Same model v2 MiniTrain, v2bs32 submitted without TTA scored only 0.822, 0825 (see Mini/my-kaggle-cassava-1-model-mini_score0822.ipynb)
#prev in my-kaggle-cassava-1-model-mini-kfold_score0825.ipynb
#latest 2/10/2021 ADD K-FOLD TO MODULE 1-MODEL MINI W/ INFERENCE + EXPORT, DELTA BS=32
#      $params: MINI = 1, DS_OVERSAMPLED = 0
#      5-fold mini modeling for rapid iteration, V1bs32 5 models
#      in MINI mode, train mini ds (not oversampled) (exported from Module 0)
#      preds,_ = learn.get_preds(dl=test_dl), no TTA
#      Score: 0.825, 0.822  Time: 48min + submission time, Rank: n/a

#history
#here 2/13/2021 MODULE 2-INFERENCE MINI EXPORTED v1bs64
#      MINI = 1 no $param
#      $params: learn = load_learner($path_model=V1bs32 5 models, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25)
#      Score: .845  submission time: <1 min+ submission time. Rank: n/a (to GH)
#      Note: Same model v2 MiniTrain, v2bs32 submitted without TTA scored only 0.822, 0825 (see Mini/my-kaggle-cassava-1-model-mini_score0822.ipynb)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import os
#import time #to track performance time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#modeling
from fastai.vision.all import *

# Input data files are available in the read-only "../input/" directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#--constants and variables

#set paths
path = Path('../input/cassava-leaf-disease-classification')

#$params
path_model = Path('../input/mycassavak5modelsmini')

#--functions
def get_x(row): return path/'train_images'/row['image_id']
def get_y(row): return row['label']


# Cassava Disease Classification - Inference
##### Objective: Classify pictures of cassava leaves into 1 of 4 disease categories or healthy.

Export saved model and create a submission.csv file, with predictions on test images.


## 0. Load K-models

- Input(s): load k-models .pkl


In [ ]:
'''was before k-folds
#track time
t0 = time.time()

#check if loaded model exists and load it
if path_model.ls(file_exts='.pkl'):
    print(path_model)
    learn_inf = load_learner(path_model/'export.pkl', cpu=False)
    #learn_inf.to_native_fp32()
else:
    print("No model to export")
'''

In [ ]:
#get k-folds
K_FOLDS = len(path_model.ls())
K_FOLDS

In [ ]:
t0 = time.time()

#check if loaded model exists and load it
if path_model.ls(file_exts='.pkl'):
    print(path_model)
    
    fold_learns_inf = [] #list of class fastai.learner.Learner
    for i in range(K_FOLDS):
        learn_inf = load_learner(path_model/f'model_{i}.pkl', cpu=False)
        #learn_inf.to_native_fp32()
        #models
        fold_learns_inf += [learn_inf]
else:
    print("No model to export")
    
len(fold_learns_inf)

## 3. Predict with K-models

In [ ]:
##Predict on one file with one model
#view test images - only one
#test_img = get_image_files(path/'test_images')
#Image.open(test_img[0])

#predict one, returns a tuple, get 0th item
#y_hat = learn_inf.predict(test_img[0])
#y_hat[0]

Test dataset

In [ ]:
#template
sample_submission = pd.read_csv(path/'sample_submission.csv')
submission = sample_submission.copy()

In [ ]:
#create test images id
test_img_id = [os.path.join(path/'test_images', x) for x in submission['image_id'].values]

#images in test files
test_img_id

In [ ]:
'''was without k-fold
test_dl = learn_inf.dls.test_dl(test_img_id)
print(len(test_dl))

#predict
#or our TTA we will be doing a 15x ensemble similar to what was performed there
preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25) #$params
#preds

#submission
submission['label'] = preds.argmax(dim=-1).numpy()
submission.head()
'''

In [ ]:
fold_preds = []
for i in range(K_FOLDS):
    test_dl = fold_learns_inf[i].dls.test_dl(test_img_id)
    #print(len(test_dl))

    #predict
    #or our TTA we will be doing a 15x ensemble similar to what was performed there
    preds, _ = fold_learns_inf[i].tta(dl=test_dl, n=4, beta=0.25) #$params
    #preds
    
    #preds
    fold_preds += [preds]
preds = torch.stack(fold_preds)

#submission
submission['label'] = preds.sum(axis=0).argmax(dim=-1).numpy() #preds.argmax(dim=-1).numpy()
submission.head()

## 4. Submit


In [ ]:
#save to csv
submission.to_csv('submission.csv', index=False)

#total inference time
t1 = time.time()
print ("Inference time (in min)", (time.time() - t0)/60)